<a href="https://colab.research.google.com/github/gyuminpk/Playground_solver/blob/main/%EC%83%89_%EB%A7%9E%EC%B6%94%EA%B8%B0_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# =============================
# Constants
# =============================

Y = 'Yellow'
R = 'Red'
B = 'Blue'

O = 'Orange'
G = 'Green'
P = 'Purple'
K = 'Black'

u = 'Up'
d = 'Down'
l = 'Left'
r = 'Right'

GRID_SIZE = 4

arrow_grid = [
    [d, d, l, u],
    [r, d, r, l],
    [d, l, d, u],
    [r, r, u, r]
]

color_grid = [
    [O, G, G, O],
    [O, K, G, K],
    [P, P, G, R],
    [P, P, K, O]
]

In [6]:

Y_BIT = 1
R_BIT = 2
B_BIT = 4
ALL_BITS = [Y_BIT, R_BIT, B_BIT]

COLOR_TO_BIT = {
    'Yellow': Y_BIT,
    'Red': R_BIT,
    'Blue': B_BIT,
    'Orange': Y_BIT | R_BIT,
    'Green': Y_BIT | B_BIT,
    'Purple': R_BIT | B_BIT,
    'Black': Y_BIT | R_BIT | B_BIT
}

BIT_TO_COLOR = {
    0: 'None',
    Y_BIT: 'Yellow',
    R_BIT: 'Red',
    B_BIT: 'Blue'
}

from copy import deepcopy

BITS = [Y_BIT, R_BIT, B_BIT]

# =============================
# Precompute influencers
# =============================

cell_influencers = [[[] for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

for ar in range(GRID_SIZE):
    for ac in range(GRID_SIZE):
        r0, c0 = ar, ac
        d0 = arrow_grid[ar][ac]

        while 0 <= r0 < GRID_SIZE and 0 <= c0 < GRID_SIZE:
            cell_influencers[r0][c0].append((ar, ac))
            if d0 == u: r0 -= 1
            elif d0 == d: r0 += 1
            elif d0 == l: c0 -= 1
            elif d0 == r: c0 += 1

# =============================
# Fast consistency check
# =============================

def partial_ok(assign):
    for r in range(GRID_SIZE):
        for c in range(GRID_SIZE):
            target = COLOR_TO_BIT[color_grid[r][c]]
            cur = 0
            unknown = False

            for ar, ac in cell_influencers[r][c]:
                if (ar, ac) in assign:
                    cur |= assign[(ar, ac)]
                else:
                    unknown = True

            # 이미 초과 → 불가능
            if cur & ~target:
                return False

            # 다 정했는데 정확히 안 맞음
            if not unknown and cur != target:
                return False
    return True

# =============================
# Backtracking
# =============================

def solve_arrows(assign, order):
    if len(assign) == len(order):
        return assign

    a = order[len(assign)]

    for bit in BITS:
        assign[a] = bit
        if partial_ok(assign):
            res = solve_arrows(assign, order)
            if res:
                return res
        del assign[a]

    return None

# =============================
# Run
# =============================

order = [(r, c) for r in range(GRID_SIZE) for c in range(GRID_SIZE)]

solution = solve_arrows({}, order)

print("✅ Arrow color assignment:")
for r in range(GRID_SIZE):
    print([BIT_TO_COLOR[solution[(r, c)]] for c in range(GRID_SIZE)])


✅ Arrow color assignment:
['Red', 'Blue', 'Yellow', 'Yellow']
['Yellow', 'Red', 'Blue', 'Yellow']
['Blue', 'Red', 'Yellow', 'Red']
['Red', 'Red', 'Blue', 'Yellow']
